In [1]:
%matplotlib tk
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os, glob, cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '../dataset/images-half/'
classes = ["100", "106", "109", "118", "217"]

In [3]:
def load_data_cnn(train_path):
    images = []
    labels = []
    classes = [lists for lists in os.walk(train_path)][0][1]    
    for cls in classes:
        file_path = os.path.join(train_path, cls, "*g")
        for file in glob.glob(file_path):
            image = cv2.imread(file)
            image = cv2.resize(image, (432, 288), cv2.INTER_LINEAR)
            image = image[288-250:288-35, 55:390]
            image = cv2.resize(image, (int(215/5), int(215/5)), cv2.INTER_LINEAR)
            label = np.zeros(len(classes))
            label[classes.index(cls)] = 1.0
            images.append(image)
            labels.append(label) 
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

dataset = load_data_cnn(train_path)

In [4]:
X_train = []
X_train.append(dataset[0][:80])
X_train.append(dataset[0][100:180])
X_train.append(dataset[0][200:280])
X_train.append(dataset[0][300:380])
X_train.append(dataset[0][400:480])
X_train = [j for i in X_train for j in i]
X_train = np.array(X_train)

y_train = []
y_train.append(dataset[1][:80])
y_train.append(dataset[1][100:180])
y_train.append(dataset[1][200:280])
y_train.append(dataset[1][300:380])
y_train.append(dataset[1][400:480])
y_train = [j for i in y_train for j in i]
y_train = np.array(y_train)

X_test = []
X_test.append(dataset[0][80:100])
X_test.append(dataset[0][180:200])
X_test.append(dataset[0][280:300])
X_test.append(dataset[0][380:400])
X_test.append(dataset[0][480:500])
X_test = [j for i in X_test for j in i]
X_test = np.array(X_test)

y_test = []
y_test.append(dataset[1][80:100])
y_test.append(dataset[1][180:200])
y_test.append(dataset[1][280:300])
y_test.append(dataset[1][380:400])
y_test.append(dataset[1][480:500])
y_test = [j for i in y_test for j in i]
y_test = np.array(y_test)

y_test_label = []
for i in range(100):
    if i >= 80:
        y_test_label.append(4)
    elif i >= 60:
        y_test_label.append(3)
    elif i >= 40:
        y_test_label.append(2)
    elif i >= 20:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
        

0 - normal <br>
1 - paced <br>
2 - lbbb <br>
3 - pvc <br>
4 - rbbb

plt.imshow(X_train[320])

In [5]:
input_shape = (int(215/5), int(215/5), 3)

In [6]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3, 3), input_shape=input_shape, activation='relu'))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5, activation='softmax'))

In [7]:
cnn_model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
classifier = cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=9, shuffle=True)

Train on 360 samples, validate on 40 samples
Epoch 1/100
360/360 [==============================] - 2s 6ms/step - loss: 12.7303 - acc: 0.1889 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/100
360/360 [==============================] - 0s 1ms/step - loss: 12.2750 - acc: 0.2250 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/100
360/360 [==============================] - 0s 1ms/step - loss: 12.3529 - acc: 0.2167 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/100
360/360 [==============================] - 0s 1ms/step - loss: 12.4515 - acc: 0.2111 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/100
360/360 [==============================] - 0s 1ms/step - loss: 12.4657 - acc: 0.2194 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/100
360/360 [==============================] - 0s 1ms/step - loss: 12.4475 - acc: 0.2167 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/100
360/360 [==============================] - 0s 1ms/step - loss: 12.4982 - acc: 0.2222 - val_loss: 16.1181 - va

360/360 [==============================] - 0s 1ms/step - loss: 0.5241 - acc: 0.8111 - val_loss: 0.1386 - val_acc: 0.9750
Epoch 61/100
360/360 [==============================] - 0s 1ms/step - loss: 0.4757 - acc: 0.8111 - val_loss: 0.1421 - val_acc: 0.9750
Epoch 62/100
360/360 [==============================] - 0s 1ms/step - loss: 0.4668 - acc: 0.8333 - val_loss: 0.1216 - val_acc: 0.9750
Epoch 63/100
360/360 [==============================] - 0s 1ms/step - loss: 0.5066 - acc: 0.8194 - val_loss: 0.1338 - val_acc: 0.9750
Epoch 64/100
360/360 [==============================] - 0s 1ms/step - loss: 0.4529 - acc: 0.8333 - val_loss: 0.0882 - val_acc: 0.9750
Epoch 65/100
360/360 [==============================] - 0s 1ms/step - loss: 0.4323 - acc: 0.8556 - val_loss: 0.0798 - val_acc: 1.0000
Epoch 66/100
360/360 [==============================] - 0s 1ms/step - loss: 0.3908 - acc: 0.8417 - val_loss: 0.0743 - val_acc: 1.0000
Epoch 67/100
360/360 [==============================] - 0s 1ms/step - loss:

In [9]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print("Loss = "+str(loss))
print("Accuracy = "+str(accuracy))

100/100 [==============================] - 0s 745us/step
Loss = 0.06385391404852271
Accuracy = 0.98


In [10]:
acc_values = cnn_model.history.history['acc']
val_acc_values = cnn_model.history.history['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'r', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [11]:
predictions = cnn_model.predict_classes(X_test, batch_size=5)

In [12]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [13]:
cn = confusion_matrix(y_test_label, predictions)

In [14]:
print ("Confusion Matrix")
cn

Confusion Matrix


array([[20,  0,  0,  0,  0],
       [ 1, 19,  0,  0,  0],
       [ 0,  0, 20,  0,  0],
       [ 1,  0,  0, 19,  0],
       [ 0,  0,  0,  0, 20]])

In [15]:
cnn_model.save("../models/cnn_5labels_8.h5")

In [17]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 41, 41, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 39, 39, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 32)        18464     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 17, 17, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
__________